In [1]:
%pip install 'pydantic-ai'

In [13]:
from google.colab import userdata
API_KEY = userdata.get('GOOGLE_API_KEY')

In [14]:

import nest_asyncio
nest_asyncio.apply()


import warnings
warnings.filterwarnings("ignore")

# Pydentic AI Agent Example

In [7]:
import asyncio
from pydantic_ai.models.gemini import GeminiModel


from dataclasses import dataclass

from pydantic import BaseModel, Field

from pydantic_ai import Agent, RunContext


class DatabaseConn:
    """This is a fake database for example purposes.

    In reality, you'd be connecting to an external database
    (e.g. PostgreSQL) to get information about customers.
    """

    @classmethod
    async def customer_name(cls, *, id: int) -> str | None:
        if id == 123:
            return 'John'

    @classmethod
    async def customer_balance(cls, *, id: int, include_pending: bool) -> float:
        if id == 123:
            return 123.45
        else:
            raise ValueError('Customer not found')


@dataclass
class SupportDependencies:
    customer_id: int
    db: DatabaseConn


class SupportResult(BaseModel):
    support_advice: str = Field(description='Advice returned to the customer')
    block_card: bool = Field(description='Whether to block their')
    risk: int = Field(description='Risk level of query', ge=0, le=10)


support_agent = Agent(
    model = GeminiModel('gemini-1.5-flash',api_key=API_KEY),
    deps_type=SupportDependencies,
    result_type=SupportResult,
    system_prompt=(
        'You are a support agent in our bank, give the '
        'customer support and judge the risk level of their query. '
        "Reply using the customer's name."
    ),
)


@support_agent.system_prompt
async def add_customer_name(ctx: RunContext[SupportDependencies]) -> str:
    customer_name = await ctx.deps.db.customer_name(id=ctx.deps.customer_id)
    return f"The customer's name is {customer_name!r}"


@support_agent.tool
async def customer_balance(
    ctx: RunContext[SupportDependencies], include_pending: bool
) -> str:
    """Returns the customer's current account balance."""
    balance = await ctx.deps.db.customer_balance(
        id=ctx.deps.customer_id,
        include_pending=include_pending,
    )
    return f'${balance:.2f}'


if __name__ == '__main__':
    deps = SupportDependencies(customer_id=123, db=DatabaseConn())
        # Get the current event loop
    loop = asyncio.get_event_loop()
    result = support_agent.run_sync('What is my balance?', deps=deps)
    print(result.data)
    """
    support_advice='Hello John, your current account balance, including pending transactions, is $123.45.' block_card=False risk=1
    """

    result = support_agent.run_sync('I just lost my card!', deps=deps)
    print(result.data)
    """
    support_advice="I'm sorry to hear that, John. We are temporarily blocking your card to prevent unauthorized transactions." block_card=True risk=8
    """

support_advice='Your balance is $123.45.' block_card=False risk=1
support_advice="John, I'm sorry to hear that you lost your card. I have blocked your card to prevent any unauthorized transactions. Please visit your nearest branch or contact us to request a new card." block_card=True risk=2


## Pydentic AI Agent with Conversations

In [19]:
import asyncio
from pydantic_ai.models.gemini import GeminiModel
from dataclasses import dataclass
from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext


class DatabaseConn:
    """This is a fake database for example purposes.

    In reality, you'd be connecting to an external database
    (e.g. PostgreSQL) to get information about customers.
    """

    @classmethod
    async def customer_name(cls, *, id: int) -> str | None:
        if id == 123:
            return 'John'

    @classmethod
    async def customer_balance(cls, *, id: int, include_pending: bool) -> float:
        if id == 123:
            return 123.45
        else:
            raise ValueError('Customer not found')


@dataclass
class SupportDependencies:
    customer_id: int
    db: DatabaseConn


class SupportResult(BaseModel):
    support_advice: str = Field(description='Advice returned to the customer')
    block_card: bool = Field(description='Whether to block their card')
    risk: int = Field(description='Risk level of query', ge=0, le=10)


support_agent = Agent(
    model=GeminiModel('gemini-1.5-flash', api_key=API_KEY),
    deps_type=SupportDependencies,
    result_type=SupportResult,
    system_prompt=(
        'You are a support agent in our bank, give the '
        'customer support and judge the risk level of their query. '
        "Reply using the customer's name. Use the conversation history to provide better context."
    ),
)


@support_agent.system_prompt
async def add_customer_name(ctx: RunContext[SupportDependencies]) -> str:
    customer_name = await ctx.deps.db.customer_name(id=ctx.deps.customer_id)
    return f"The customer's name is {customer_name!r}"


@support_agent.tool
async def customer_balance(
    ctx: RunContext[SupportDependencies], include_pending: bool
) -> str:
    """Returns the customer's current account balance."""
    balance = await ctx.deps.db.customer_balance(
        id=ctx.deps.customer_id,
        include_pending=include_pending,
    )
    return f'${balance:.2f}'


@support_agent.tool
async def new_card_request(
    ctx: RunContext[SupportDependencies], include_pending: bool
) -> str:
    """Returns reture new card request."""
    request_no = "abc123"
    return f'request no {request_no}'

if __name__ == '__main__':
    deps = SupportDependencies(customer_id=123, db=DatabaseConn())
    conversation_history = []  # To store the conversation history

    print("Welcome to the Bank Support Chatbot!")
    print("Type 'exit' to end the chat.\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Goodbye! Have a great day!")
            break

        # Add user input to the conversation history
        conversation_history.append(f"User: {user_input}")

        # Combine conversation history into a single input string
        combined_input = "\n".join(conversation_history)

        # Run the agent synchronously with the combined input
        try:
            result = support_agent.run_sync(combined_input, deps=deps)

            # Add the agent's response to the conversation history
            conversation_history.append(f"Agent: {result.data.support_advice}")

            print(f"Support Agent: {result.data.support_advice}")
            print(f"is card Blocked : {result.data.block_card}  | Risk Level : {result.data.risk}")
        except Exception as e:
            print(f"Support Agent: Sorry, I couldn't process your request. Error: {e}\n")

Welcome to the Bank Support Chatbot!
Type 'exit' to end the chat.

You: Hello
Support Agent: Hello John, how can I help you today?
is card Blocked : False  | Risk Level : 0
You: Can you block my card
Support Agent: Your card has been blocked. Please contact us to request a new card.
is card Blocked : True  | Risk Level : 5
You: I want to new card
Support Agent: Your new card request has been received. You will receive your new card in 5-7 business days.  
is card Blocked : False  | Risk Level : 1
You: what is request id
Support Agent: Your new card request (abc123) has been received. You will receive your new card in 5-7 business days.
is card Blocked : False  | Risk Level : 1
You: exit
Goodbye! Have a great day!
